In [ ]:
# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X = X_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()

print('Shape of label tensor:', y.shape)
print('Shape of X:', X.shape)
print('Shape of embeddings_GloVe:', embeddings_GloVe.shape)

In [ ]:
tf.random.set_seed(42)

#batch_size = None
#batch_size=64

model = Sequential()
model.add(Input(shape=(20,)))
# Adding the Embedding layer with pre-trained weights and specifying input_length
model.add(Embedding(input_dim=embeddings_GloVe.shape[0],
                    output_dim=embeddings_GloVe.shape[1],
                    weights=[embeddings_GloVe],
                    trainable=False))

model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
# It's common to add a MaxPooling layer after a Conv layer to reduce dimensionality
model.add(MaxPooling1D())

model.add(Bidirectional(LSTM(64, # Number of hidden states, number of reccurent units for each vector
    activation='tanh',
    recurrent_activation='sigmoid',
    recurrent_dropout=0.2,
    return_sequences=False)))

model.add(Dense(77, activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)


# history = model.fit(X_train_LSTM, y_train_LSTM,
#                     epochs=20, # Adjust based on your needs
#                     validation_split=0.2,
#                     batch_size=64,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
